In [1]:
"""
Import modules
"""

import pathlib
import numpy as np

import SimpleITK as sitk
import matplotlib.pyplot as plt

from platipy.imaging import ImageVisualiser
from platipy.imaging.label.utils import get_com

from platipy.imaging.registration.linear import linear_registration
from platipy.imaging.registration.deformable import fast_symmetric_forces_demons_registration
from platipy.imaging.registration.utils import apply_transform

%matplotlib notebook

In [46]:
def getPETseg(masked_pet_breast,img_pt,pt_id,timepoint,tumour_path):
    mask_arr=sitk.GetArrayFromImage(masked_pet_breast)
    mask_arr=mask_arr.flatten() 

    p = np.percentile(mask_arr[mask_arr>0], 98)
    tum = sitk.Mask(img_pt, masked_pet_breast>p)
    tum = sitk.Cast(tum, sitk.sitkInt64)
    tum_cc = sitk.RelabelComponent(sitk.ConnectedComponent(tum))
    tum = (tum_cc==1)
    tum=sitk.Resample(tum,img_pt)
    sitk.WriteImage(tum, tumour_path+f"WES_{pt_id}_TIMEPOINT_{timepoint}_PET_TUMOUR.nii.gz")

    return(masked_pet_breast,tum)

def RunPETseg():
    PET_timepoints={"04": ["0","1","2"],"05":["0","1","2"],"06":["0","1","2"],"07":["0","1","2"],"08":["0","1","2"],
    "09":["0","1","2"],"10":["0","1","2"],"12":["0","1","2"],"13":["0","1","2"],"14":["0","1","2"],"15":["0","1","2"],"16":["0","1","2"],
    "18":["0","1","2"],"19":["0","1"]}

    """
    Set up data
    """
    data_dir = pathlib.Path(f"/home/alicja/PET_LAB_PROCESSED")

    #pt_id_list = sorted([i.name[4:] for i in data_dir.glob("WES*")])
    pt_id_list=["004"]
    timepoints=[1,2,3]

    for pt_id in pt_id_list:
        for timepoint in timepoints:
            """
            Define file names
            """

            breast_contour_fp="/home/alicja/PET-LAB Code/PET-LAB/PET segmentation/PET breast contours/"
            filename_breast_contour= breast_contour_fp+f"breast_contour_dilate_{pt_id[-2:]}_{PET_timepoints[pt_id[-2:]][timepoint-1]}.nii.gz"
            filename_pt = str(data_dir / f"WES_{pt_id}" / "IMAGES" / f"WES_{pt_id}_TIMEPOINT_{timepoint}_PET.nii.gz")

            tumour_path="/home/alicja/PET-LAB Code/PET-LAB/PET segmentation/New PET tumours/"

            img_breast = sitk.ReadImage( filename_breast_contour )
            img_pt = sitk.ReadImage( filename_pt )

            """
            Resample Breast Mask
            """
            img_breast_res = sitk.Resample(
                img_breast,
                img_pt,
                sitk.Transform(),
                sitk.sitkNearestNeighbor
            )

            """
            Input data plots
            """
            plots_path="/home/alicja/PET-LAB Code/PET-LAB/PET segmentation/"

            vis = ImageVisualiser(img_pt, cut=get_com(img_breast_res), figure_size_in=5, window=[0,1])
            vis.add_contour(img_breast_res)
            fig = vis.show()
            fig.savefig(plots_path+f"PLOTS/WES_{pt_id}_TIMEPOINT_{timepoint}_PETSEG_0.jpeg",dpi=400)

            """
            Mask PET with breast mask / run tumour segmentation
            """

            masked_pet_breast = sitk.Mask(img_pt, img_breast_res)
            masked_pet_breast,tum=getPETseg(masked_pet_breast,img_pt,pt_id,timepoint,tumour_path)

            vis = ImageVisualiser(img_pt, cut=get_com(tum), figure_size_in=5, window=[0,1])
            vis.add_contour(tum)
            fig = vis.show()
            fig.savefig(plots_path+f"PLOTS/WES_{pt_id}_TIMEPOINT_{timepoint}_PETSEG_1.jpeg",dpi=400)

            print(f"Patient {pt_id} time point {timepoint} PET segmentation complete")

In [6]:
"""
Need to edit some breast masks 
12 TP3 - need to erode maybe by 3 pixels to remove skin
15 TP1 - need to dilate by 5 pixels?
19 TP2 - need to dilate by 4 pixels?
"""

#breask_mask_12_2=sitk.ReadImage("/home/alicja/PET-LAB Code/PET-LAB/PET segmentation/PET breast contours/breast_contour_dilate_12_2_old.nii.gz", sitk.sitkUInt64)
#breask_mask_15_0=sitk.ReadImage("/home/alicja/PET-LAB Code/PET-LAB/PET segmentation/PET breast contours/breast_contour_dilate_15_0_old.nii.gz", sitk.sitkUInt64)
#breask_mask_19_1=sitk.ReadImage("/home/alicja/PET-LAB Code/PET-LAB/PET segmentation/PET breast contours/breast_contour_dilate_19_1_old.nii.gz", sitk.sitkUInt64)

#mask_12_2=sitk.BinaryErode(breask_mask_12_2,(3,3,3))
#mask_15_0=sitk.BinaryDilate(breask_mask_15_0,(5,5,5))
#mask_19_1=sitk.BinaryDilate(breask_mask_19_1,(3,3,3))

#sitk.WriteImage(mask_12_2,"/home/alicja/PET-LAB Code/PET-LAB/PET segmentation/PET breast contours/breast_contour_dilate_12_2.nii.gz")
#sitk.WriteImage(mask_15_0,"/home/alicja/PET-LAB Code/PET-LAB/PET segmentation/PET breast contours/breast_contour_dilate_15_0.nii.gz")
#sitk.WriteImage(mask_19_1,"/home/alicja/PET-LAB Code/PET-LAB/PET segmentation/PET breast contours/breast_contour_dilate_19_1.nii.gz")

In [44]:
"""
Need to register breast mask for PT 16 TP 1
"""
breast_contour_16_0=sitk.ReadImage("/home/alicja/PET_LAB_PROCESSED/WES_016/LABELS/WES_016_RTSIM_LABEL_WHOLE_BREAST_CTV.nii.gz")
rt_sim=sitk.ReadImage("/home/alicja/PET_LAB_PROCESSED/WES_016/IMAGES/WES_016_CT_RTSIM.nii.gz")
pet_16_1=sitk.ReadImage("/home/alicja/PET_LAB_PROCESSED/WES_016/IMAGES/WES_016_TIMEPOINT_1_PET.nii.gz")
ct_ac_16_1=sitk.ReadImage("/home/alicja/PET_LAB_PROCESSED/WES_016/IMAGES/WES_016_TIMEPOINT_1_CT_AC.nii.gz")
ct_array=sitk.GetArrayFromImage(ct_ac_16_1)
ct_array[:,:151,:]=0
ct=sitk.GetImageFromArray(ct_array)
ct.CopyInformation(ct_ac_16_1)

ct=sitk.Resample(ct,pet_16_1)
breast=sitk.Resample(breast_contour_16_0,pet_16_1)

rtsim_to_ct, tfm_rtsim_to_ct = linear_registration(
    ct,
    rt_sim,
    shrink_factors= [10,5],
    smooth_sigmas= [2,1],
    sampling_rate= 1,
    final_interp= 2,
    metric= 'mean_squares',
    optimiser= 'gradient_descent_line_search',
    reg_method='rigid'
)

breast=apply_transform(
    breast,
    ct,
    tfm_rtsim_to_ct
)

sitk.WriteImage(breast,"/home/alicja/PET-LAB Code/PET-LAB/PET segmentation/PET breast contours/breast_contour_dilate_16_0.nii.gz")

"\nThe above code doesn't work too well\n"

In [47]:
RunPETseg()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Patient 004 time point 1 PET segmentation complete


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Patient 004 time point 2 PET segmentation complete


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Patient 004 time point 3 PET segmentation complete
